In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [6]:
model = ChatGroq(
    model_name="openai/gpt-oss-120b",
    api_key=GROQ_API_KEY,
    temperature=0.7
)

In [43]:
class BatsmanState(TypedDict):
    name:str
    runs:int
    balls:int
    fours:int
    sixes:int

    sr:float
    bpb:float
    boundary_percent:float

In [44]:
def calculate_sr(state:BatsmanState)->dict:
    sr = (state['runs']/state['balls'])*100
    return {'sr':sr}

In [45]:
def calculate_bpb(state:BatsmanState)->dict:
    bpb = state['balls']/(state['fours']+state['sixes'])
    return {'bpb':bpb}

In [46]:
def calculate_boundary_percent(state:BatsmanState)->dict:
    boundary_percent = (((state['fours']*4) + (state['sixes']*6))/state['runs'])*100
    return {'boundary_percent':boundary_percent}

In [47]:
def summary(state:BatsmanState)->dict:
    summary = f"""Provide a summary of the batsman's performance based on the following statistics:
        Name:{state['name']}
        Runs:{state['runs']}
        Balls:{state['balls']}
        Fours:{state['fours']}
        Sixes:{state['sixes']}
        Strike Rate:{state['sr']}
        Balls per Boundary:{state['bpb']}
        Boundary Percentage:{state['boundary_percent']}
        """
    return {'summary':summary}


In [57]:
graph=StateGraph(BatsmanState,model=model)

# add nodes
graph.add_node('calculate_sr',calculate_sr)
graph.add_node('calculate_bpb',calculate_bpb)
graph.add_node('calculate_boundary_percent',calculate_boundary_percent)
graph.add_node('summary',summary)

# add edges
graph.add_edge(START,'calculate_sr')
graph.add_edge(START,'calculate_bpb')
graph.add_edge(START,'calculate_boundary_percent')

graph.add_edge('calculate_sr','summary')
graph.add_edge('calculate_bpb','summary')
graph.add_edge('calculate_boundary_percent','summary')

graph.add_edge('summary',END)

workflow = graph.compile()


In [58]:
initial_state={
    'name':'Virat Kohli',
    'runs':100,
    'balls':75,
    'fours':7,
    'sixes':2  
}

In [59]:
final_state = workflow.invoke(initial_state)
print(final_state)

{'name': 'Virat Kohli', 'runs': 100, 'balls': 75, 'fours': 7, 'sixes': 2, 'sr': 133.33333333333331, 'bpb': 8.333333333333334, 'boundary_percent': 40.0}
